<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_TL3_Scaling_Up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Transfer Learning Scaling Up

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import Model, Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers import RandomFlip, RandomHeight, RandomRotation, RandomWidth, RandomZoom
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory

## Setup

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-darkgrid')

### Constantes

In [ ]:
SEMENTE = 2008193

DIR_TREINO = '101_food_classes_10_percent/train'
DIR_TESTE  = '101_food_classes_10_percent/test'

TAMANHO_LOTE = 32
FORMATO_IMAGEM = (224, 224)
MODO_CLASSE = 'categorical'

FORMATO_ENTRADA = FORMATO_IMAGEM + (3,)
ATIVACAO = 'softmax'

PERDA = 'categorical_crossentropy'
METRICAS = ['accuracy']

# OTIMIZADOR = 'Adam'
APRENDIZADO = 0.001
APRENDIZADO_RESSINTONIZADO = 0.0001

ITERACOES = 5
ITERACOES_RESSINTONIZADAS = ITERACOES + 5

### Funções

In [ ]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

In [ ]:
from funcoes import grafico_historico_por_iteracao, grafico_historicos_complementares, unzip_data

## Dados

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

unzip_data('101_food_classes_10_percent.zip')

### Importando os dados e tranformando-os em lotes

In [ ]:
dados_treino = image_dataset_from_directory(directory=DIR_TREINO,
                                            batch_size=TAMANHO_LOTE,
                                            image_size=FORMATO_IMAGEM,
                                            label_mode=MODO_CLASSE,
                                            seed=SEMENTE)

dados_teste = image_dataset_from_directory(directory=DIR_TESTE,
                                           batch_size=TAMANHO_LOTE,
                                           image_size=FORMATO_IMAGEM,
                                           label_mode=MODO_CLASSE,
                                           seed=SEMENTE)

In [ ]:
rotulos = dados_treino.class_names

## Pré-modelagem

### Expansão dos dados

In [ ]:
expansao_dados = Sequential(name='expansao_dados')

expansao_dados.add(RandomFlip('horizontal'))
expansao_dados.add(RandomHeight(0.2))
expansao_dados.add(RandomRotation(0.2))
expansao_dados.add(RandomWidth(0.2))
expansao_dados.add(RandomZoom(0.2))

### Modelo-base

In [ ]:
modelo_base = EfficientNetB0(include_top=False)
modelo_base.trainable = False

## Modelo inicial

In [ ]:
entradas = Input(shape=FORMATO_ENTRADA, name='camada_entrada')

# Há um bug na versão 2.8 do TensorFlow que faz necessário forçar o treinamento para que a expansão dos dados funcione.
expandidos = expansao_dados(entradas, training=True)

camadas = modelo_base(expandidos, training=False)
camadas = GlobalAveragePooling2D(name='agrupamento_media_global')(camadas)

saidas = Dense(len(rotulos), activation=ATIVACAO, name='camada_saida')(camadas)

modelo = Model(inputs=entradas, outputs=saidas, name='Modelo')

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

In [ ]:
historico_inicial = modelo.fit(dados_treino,
                               epochs=ITERACOES,
                               steps_per_epoch=len(dados_treino),
                               validation_data=dados_teste,
                               validation_steps=int(0.15 + len(dados_teste)), # Apenas 15% da base de teste
                               verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico_inicial)

In [ ]:
validacao_inicial = modelo.evaluate(dados_teste)
validacao_inicial

## Ressintonizando modelo

In [ ]:
# "Descongela" as últimas 5 camadas do modelo_base (EfficientNetB0).
for layer in modelo_base.layers[-5:]:
    layer.trainable = True

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO_RESSINTONIZADO),
               metrics=METRICAS)

In [ ]:
historico_ressintonizado = modelo.fit(dados_treino,
                                     epochs=ITERACOES_RESSINTONIZADAS,
                                     steps_per_epoch=len(dados_treino),
                                     initial_epoch=len(historico_inicial.epoch),
                                     validation_data=dados_teste,
                                     validation_steps=int(0.15 * len(dados_teste)),
                                     verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico_ressintonizado)

In [ ]:
grafico_historicos_complementares(historico_inicial, historico_ressintonizado)

In [ ]:
validacao_ressintonizada = modelo.evaluate(dados_teste)
validacao_ressintonizada

## Salvando modelo

In [ ]:
# Formato HDF5
modelo.save('scaling_up_HDF5.h5')

## Carregando modelo

In [ ]:
# modelo = tf.keras.models.load_model('scaling_up_HDF5.h5')

## Previsão

In [ ]:
previsao = modelo.predict(dados_teste, verbose=1)

In [ ]:
print("Exemplo de previsões")
print(f"Quantidade de previsões: {len(previsao[0])}")
print(f"Soma das previsões: {sum(previsao[0])}")
print(f"Previsões:\n{previsao[0]}")
print(f"Maior probabilidade: {(previsao[0].max() * 100):0.2f}% ({rotulos[previsao[0].argmax()]})")